In [1]:
from azureml.core import Workspace
from azureml.core import Experiment, Environment
from azureml.pipeline.core import Pipeline
from azureml.data.data_reference import DataReference
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.runconfig import RunConfiguration
from modules.ingestion.data_ingestion_step import data_ingestion_step
from modules.preprocess.data_preprocess_step import data_preprocess_step
from modules.train.train_step import train_step
from modules.evaluate.evaluate_step import evaluate_step
from modules.explainer.explainer_step import explainer_step
from modules.deploy.deployTest_step import deployTest_step
from modules.deploy.deployProd_step import deployProd_step
from azureml.pipeline.core import PipelineData

import json

In [2]:
ws = Workspace.from_config()

datastore = ws.get_default_datastore()

datastore = DataReference(datastore, mode='mount')

In [3]:
# Create CPU compute target
print('Creating CPU compute target ...')
cpu_cluster_name = 'cpuclst'
cpu_compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2', 
                                                           idle_seconds_before_scaledown=1200,
                                                           min_nodes=0, 
                                                           max_nodes=3)
cpu_compute_target = ComputeTarget.create(ws, cpu_cluster_name, cpu_compute_config)
cpu_compute_target.wait_for_completion(show_output=True)


adb_compute_target = ComputeTarget(workspace=ws, name='databricks')


aml_run_config = RunConfiguration()
aml_run_config.target = cpu_compute_target
curated_environment = Environment.get(workspace=ws,name="sklearn_22_2")# name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
aml_run_config.environment = curated_environment

Creating CPU compute target ...


In [4]:
# Step 1: Data ingestion Azure Databricks
data_ingestion_step, data_ingestion_outputs = data_ingestion_step(datastore, adb_compute_target)

# Step 2: Data preprocessing 
data_preprocess_step, data_preprocess_outputs = data_preprocess_step(data_ingestion_outputs, cpu_compute_target,aml_run_config)

# Step 3: Train Model
train_step, train_outputs = train_step(data_preprocess_outputs['train_dir'], cpu_compute_target,aml_run_config)

# Step 4: Evaluate Model
evaluate_step, evaluate_outputs = evaluate_step(train_outputs['model_dir'], data_preprocess_outputs['test_dir'], cpu_compute_target,aml_run_config)

# Step 5: Evaluate Model
explainer_step = explainer_step(train_outputs['model_dir'], data_preprocess_outputs['test_dir'],data_preprocess_outputs['train_dir'], cpu_compute_target,aml_run_config)

# Step 6: Deploy Model Test
deploy_test_step, deploytest_outputs = deployTest_step(train_outputs['model_dir'],evaluate_outputs['accuracy_file'], data_preprocess_outputs['test_dir'], cpu_compute_target,aml_run_config)

# Step 7: Deploy Model Prod
#deploy_prod_step, deployprod_outputs = deployProd_step( evaluate_outputs['accuracy_file'], data_preprocess_outputs['test_dir'],cpu_compute_target,aml_run_config)


In [5]:
# Submit pipeline
print('Submitting pipeline ...')
pipeline_parameters = {
    'n_estimators': 50 
}
pipeline_parameters_old = {
    'max_depth': 5,
    'n_estimators': 500,
    'prod_deploy': 0,
    'endpoint_traffic_pct': 10,
    'set_endpoint_as_default': 0
}


# Submit pipeline
print('Submitting pipeline ...')

pipeline = Pipeline(workspace=ws, steps=[data_ingestion_step,data_preprocess_step,train_step,evaluate_step,explainer_step,deploy_test_step])
pipeline_run = Experiment(ws, 'churn-pipeline').submit(pipeline,pipeline_parameters=pipeline_parameters,regenerate_outputs=True)

Submitting pipeline ...
Submitting pipeline ...
Created step adb-churn-ingest [db6528fe][1170fc73-0aef-40c1-9cac-a1ddb3a6ae38], (This step will run and generate new outputs)Created step data_preprocess.py [a43b9105][4f026527-3442-4f2b-a60b-93248177ddf4], (This step will run and generate new outputs)

Created step train_data [d655df6d][895e6764-598b-49d1-aade-4b0046ddf7e8], (This step will run and generate new outputs)
Created step eval [9a807506][e8dc1bda-25c6-4a2b-8f85-5ccfdfc0586d], (This step will run and generate new outputs)
Created step explainer [7d647fff][f8aedc6d-4f56-466f-a074-663b579fd370], (This step will run and generate new outputs)
Created step deployTest.py [e38b6afe][e1a1c6dc-0158-4d69-94ab-cc510a27c009], (This step will run and generate new outputs)
Created data reference workspaceblobstore for StepId [2a0bfb5e][11d66f8c-beb8-4715-b46b-f45746c60518], (Consumers of this data will generate new runs.)
Submitted PipelineRun e8b605a0-216f-4d2f-816b-215938402229
Link to Azu